In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import ipywidgets as widgets
import math

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from math import comb
from utils.paralell import (create_gambles, create_gamble_pairs, create_mag_thrs, 
                            create_dmag_thrs, create_var_thrs, is_nobrainer, 
                            is_g_win, is_g_loss, is_g_mixed, is_mixed, 
                            wealth_change, shuffle_along_axis)
from utils.style import rc_style

### Settings

- `n_fractals`: number of distinct wealth changes
- `n_trials`: experiment duration
- `x0`: initial endowment

In [2]:
n_fractals = 9
n_trials = 360
x0 = 1000

### Functions definition

In [3]:
mpl.rcParams.update(rc_style)

# color definition
cmap_rdylgn = mpl.cm.get_cmap("RdYlGn_r")
cmap_cool = mpl.cm.get_cmap("cool")

WIN_DARK = cmap_rdylgn(0.001) 
WIN_LIGHT = cmap_rdylgn(0.15)
LOSS_DARK = cmap_rdylgn(0.999)
LOSS_LIGHT = cmap_rdylgn(0.85)
MIX_LIGHT = "gold"
MIX_DARK = "goldenrod"
VAR_LOWER = cmap_cool(0.33)
VAR_UPPER = cmap_cool(0.67)
NEUTRAL_LIGHT = "silver"
WHITE = "white"

def gamble_color(g):
    """Determine dot color given gamble type.
    
    Args:
        g (np.array):
            Gamble array of shape (2, 0).
    
    Returns:
        Matplotlib-friendly color.
    """
    if np.all(g > 0):
        color = WIN_DARK
    elif np.all(g < 0):
        color = LOSS_DARK
    elif np.all(g == 0):
        color = WHITE
    elif np.any(g == 0) and np.any(g > 0):
        color = WIN_LIGHT
    elif np.any(g == 0) and np.any(g < 0):
        color = LOSS_LIGHT
    else:
        color = MIX_LIGHT
    return color 

def gamble_pair_color(gp):
    """Determine edge color given gamble pair type.
    
    Args:
        gp (np.array):
            Gamble pair array of shape (2, 2).
    
    Returns:
        Tuple of matplotlib-friendly color and zorder used to move in front 
        colored edges.
    """
    if np.all(gp >= 0):
        color = WIN_DARK
        zorder = 1
    elif np.all(gp <= 0):
        color = LOSS_DARK
        zorder = 1
    elif is_mixed(gp):
        color = MIX_DARK
        zorder = 1
    else:
        color = NEUTRAL_LIGHT
        zorder = -1
    return color, zorder

def split_gamble_pairs_types(gamble_pairs):
    """Sort gambles into different types producing summary table.
    
    Args:
        gamble_pairs (list):
            List of (2, 2) arrays. Each array correspond to gamble pair.
            
    Returns:
        Table represented as (3, 3) array. Rows and columns correspond to win,
        mixed and loss gambles. For example entry [0, 1] reflects number of 
        gamble pairs composed of one win and one mixed gambles. Table is 
        symmetric.   
    """
    gp_tab = np.zeros((3, 3))
    is_fcns = [is_g_win, is_g_mixed, is_g_loss]

    for gp in gamble_pairs:
        for i, fi in enumerate(is_fcns):
            for j, fj in enumerate(is_fcns):
                if fi(gp[0]) and fj(gp[1]):
                    gp_tab[i, j] += 1

    gp_tab = gp_tab + gp_tab.T
    gp_tab[np.diag_indices_from(gp_tab)] /= 2
    gp_tab = gp_tab.astype(int)
    return gp_tab

def plot_space(n_fractals, ml_idx=0, mh_idx=0, md_idx=0, vl_idx=0, vh_idx=0, 
               exclude_nobrainers=False, c_rse=100_000, c_lin=100, c_log=0.1,
               n_sim=20, btn=None):
    """Shows restricted gamble-pair space along with final wealth distribution.
    
    This function is used to visualize gamble and gamble-pair spaces which are 
    geometrically restricted using bounds for average growth rate, gamble 
    variance and gamble distance in average growth rate dimension. Resulting 
    subspace is summarized by showing distribution of final wealth for three
    realizations (for dynamical risk attitudes -1, 0 and 1). Additional 
    information contain total number of gambles, gamble pairs and distribution
    of gamble pair types.
    
    Args:
        n_fractals (int):
            Number of available wealth changes.
        ml_idx (int):
            Index for lower avg. growth rate threshold. Increasing this excludes
            worst gambles.
        mh_idx (int):
            Index for upper avg. growth rate threshold. Increasing this excludes
            most profitable gambles.    
        md_idx (int):
            Index for avg. growth rate distance threshold. For 0, no threshold
            is applied. Increasing this excludes easiest choices.
        vl_idx (int):
            Index for lower variance threshold. Increasing this excludes gambles
            with similar growth rates. Specifically, setting vl_idx to 1 results
            in exclusion of deterministic gambles.
        vh_idx (int):
            Index for upper variance threshold. Increasing this excludes gambles
            with distant growth rates.
        exclude_nobrainers (bool):
            Decision value if no-brainer gamble pairs should be excluded. 
            No-brainer gamble pair contains two gambles with at least one 
            repeating fractal
        c_rse (float):
            Scaling factor for max growth rate in risk-seeking dynamics.
        c_lin (float):
            Scaling factor for max growth rate in additive dynamics.
        c_log (float):
            Scaling factor for max growth rate in multiplicativ dynamics.
        n_sim (int):
            Number of "micro-simulations" used to generate final wealth 
            distribution. Increasing this, increases granularity and precision
            of final distribution.            
        btn (defaults to None):
            Not used hack variable for refreshing interactive plot.
            
    Returns:
        None. Function results in drawing a figure.
    """
    # Calculate bounds levels
    c = 1
    mag_thrs = create_mag_thrs(c, n_fractals)
    var_thrs = create_var_thrs(c, n_fractals)
    dmag_thrs = create_dmag_thrs(c, n_fractals)
    
    ml_bound = mag_thrs[ml_idx]
    mh_bound = mag_thrs[-1 - mh_idx]
    md_bound = dmag_thrs[-1 - md_idx]
    vl_bound = var_thrs[vl_idx]
    vh_bound = var_thrs[-1 - vh_idx]

    # Creaate and filter gambles and gamble pairs
    gambles = create_gambles(c, n_fractals)
    gambles = [
        g for g in gambles 
        if np.mean(g) > ml_bound
        and np.mean(g) < mh_bound
        and np.abs(g[0] - g[1]) > vl_bound
        and np.abs(g[0] - g[1]) < vh_bound
    ]

    gamble_pairs = create_gamble_pairs(gambles)
    gamble_pairs = [
        gp for gp in gamble_pairs 
        if np.abs(np.mean(gp[0]) - np.mean(gp[1])) < md_bound
    ]
    if exclude_nobrainers:
        gamble_pairs = [
            gp for gp in gamble_pairs
            if not is_nobrainer(gp)
        ]

    # Figure drawing
    fig, ax = plt.subplots(figsize=(12, 12), facecolor="w")

    ax.plot([-c, c], [-c, c], c="k", lw=2)
    ax.set_xlim([-c, c])
    ax.set_ylim([c, -c])
    ax.set_xticks(np.linspace(-c, c, n_fractals))
    ax.set_yticks(np.linspace(-c, c, n_fractals))
    ax.tick_params(axis='both', which='major', pad=10)
    
    # Gambles & gamble pairs
    for gp in gamble_pairs:    
        gp_color, gp_zorder = gamble_pair_color(gp)
        line = ax.plot(
            gp[:, 0], 
            gp[:, 1], 
            color=gp_color,
            zorder=gp_zorder,
            alpha=0.85, 
        )
    for g in gambles:        
        dot = ax.plot(
            g[0],
            g[1],
            marker="o", 
            ms=15, 
            mfc=gamble_color(g), 
            mec="k",
            mew=2,
            zorder=99
        )
        # overlay dots over axis
        for d in dot:
            d.set_clip_on(False)

    # Line bounds
    if ml_idx != 0:
        ax.plot([-c, ml_bound], [2*ml_bound + c, ml_bound], 
                c=LOSS_DARK, lw=5, ls=":")
    if mh_idx != 0:
        ax.plot([2*mh_bound - c, mh_bound], [c, mh_bound], 
                c=WIN_DARK, lw=5, ls=":")
    if vl_idx != 0:
        ax.plot([-c, c - vl_bound], [-c + vl_bound, c], 
                c=VAR_LOWER, lw=5, ls=":")
    if vh_idx != 0:
        ax.plot([-c, c - vh_bound], [-c + vh_bound, c], 
                c=VAR_LOWER, lw=5, ls=":")

    # Arrow bounds    
    if md_idx != 0:
        offset = c / 50
        d = md_bound / np.sqrt(8)
        ax.annotate(
            text="",
            xy=(-d + offset, -d - offset), 
            xytext=(d + offset, d - offset), 
            arrowprops=dict(lw=3, color=LOSS_DARK, arrowstyle='<->'),
            annotation_clip=False
        )
    
    # Wealth distribution (inset figure)
    n_gp = len(gamble_pairs)
    if n_gp:
        repetition_factor = math.ceil(n_trials / n_gp)

        x0_vec = x0 * np.ones(n_sim) 
        gambles_optimal = [gp[np.argmax(np.mean(gp, axis=1))] 
                           for gp in gamble_pairs * repetition_factor]
        gambles_optimal = np.stack(gambles_optimal, axis=1).T
        gambles_optimal = shuffle_along_axis(gambles_optimal, axis=0)[:n_trials]
        coin_toss = np.random.randint(0, 2, (n_sim, n_trials))
        growth_rates_sampled = gambles_optimal[np.arange(n_trials), coin_toss]
        growth_rates_sum = np.sum(growth_rates_sampled, axis=1)

        ins = ax.inset_axes([0.425, 0.685, 0.575, 0.315])

        kde_iteritems = [
            (c_rse, -1, "-1", WIN_DARK), 
            (c_lin, 0, "+", MIX_DARK), 
            (c_log, 1, "×", LOSS_DARK)
        ]

        for c_dynamic, eta_dynamic, label, color in kde_iteritems:
            wc = wealth_change(
                x0_vec, 
                growth_rates_sum * c_dynamic, 
                eta_dynamic
            ) 
            if not math.isclose(np.var(wc), 0):
                sns.kdeplot(
                    wc, 
                    ax=ins, 
                    shade=True, 
                    linewidth=2, 
                    bw_adjust=0.25,
                    alpha=0.25,
                    color=color,
                    label=label,
                    cut=0
                )

        ins.set_xlabel(r"wealth")    
        ins.set_yticks([])
        ins.set_ylabel("Density")
        ins.legend(bbox_to_anchor=(0, 1.05))
        ins.yaxis.set_label_coords(-0.025, 0.18)
    
    # Number and percentage annotations
    n_all_gambles = comb(n_fractals, 2) + n_fractals
    n_all_pairs = comb(n_all_gambles, 2)
    perc_gambles = len(gambles) / n_all_gambles * 100
    perc_pairs = len(gamble_pairs) / n_all_pairs * 100
    text_gambles = fr"$\#$ga: {len(gambles)} ({perc_gambles:.1f}\%)"
    text_pairs = fr"$\#$gp: {len(gamble_pairs)} ({perc_pairs:.1f}\%)" 
    ax.text(c/2, -c/10 - c/20, 
            text_gambles, size=25, usetex=True, ha="left", va="top")
    ax.text(c/2, -c/20, 
            text_pairs, size=25, usetex=True, ha="left", va="top")
    
    # Table
    gp_tab = split_gamble_pairs_types(gamble_pairs)
    table = r"\begin{tabular}{ c | c | c | c } & $+$ & $\pm$ & $-$ " \
          + fr"\\\hline $+$ & {gp_tab[0, 0]} & {gp_tab[0, 1]} & {gp_tab[0, 2]}" \
          + fr"\\\hline $\pm$ & {gp_tab[1, 0]} & {gp_tab[1, 1]} & {gp_tab[1, 2]}" \
          + fr"\\\hline $-$ & {gp_tab[2, 0]} & {gp_tab[2, 1]} & {gp_tab[2, 2]}" \
          + r"\end{tabular}"
    ax.text(c, c/3 - c/15, table, size=25, usetex=True, va="center", ha="right")

In [4]:
interact(
    plot_space, 
    n_fractals=widgets.fixed(n_fractals),
    ml_idx=widgets.IntSlider(
        min=0, max=2*n_fractals-1, step=1, value=0, description="Lower γ"),
    mh_idx=widgets.IntSlider(
        min=0, max=2*n_fractals-1, step=1, value=0, description="Upper γ"),
    md_idx=widgets.IntSlider(
        min=0, max=2*n_fractals-1, step=1, value=0, description="Δγ"),
    vl_idx=widgets.IntSlider(
        min=0, max=n_fractals, step=1, value=1, description="Lower var"),
    vh_idx=widgets.IntSlider(
        min=0, max=n_fractals, step=1, value=0, description="Upper var"),
    c_rse=widgets.FloatSlider(
        min=1000, max=1_000_000, step=1000, value=100_000, 
        description="$c_{\mathrm{rse}}$"),
    c_lin=widgets.FloatSlider(
        min=0.5, max=500, step=0.5, value=50, 
        description="$c_{\mathrm{lin}}$"),
    c_log=widgets.FloatSlider(
        min=0.001, max=0.1, step=0.001, value=0.01, 
        description="$c_{\mathrm{log}}$", 
        readout_format='.3f'),
    n_sim=widgets.IntSlider(
        min=10, max=2000, step=10, value=100, description="simulations"),
    exclude_nobrainers=widgets.Checkbox(
        value=True, description='Exclude nobrainers'),
    btn = widgets.Checkbox(description="Refresh")
);

interactive(children=(IntSlider(value=0, description='Lower γ', max=17), IntSlider(value=0, description='Upper…